In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

### os
import os
import sys

### datetimes
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

### scipy
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import cartopy.crs as ccrs
import dask
from dask.diagnostics import ProgressBar
from tqdm import tqdm

### plotting
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns

In [3]:
import pathlib

HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd()

### import local functions for the processing of the C3S forecasts 

In [4]:
sys.path.append('../..')

In [5]:
from ICU_Water_Watch import C3S, domains, geo, utils

### parameters for papermill

In [6]:
# Parameters
quantiles = 'tercile'
country = "Kiribati"
admin = "Gilberts-North"
lag = 1

### list of GCMs 

In [7]:
list_GCMs = ['ECMWF','UKMO','METEO_FRANCE','CMCC','DWD', 'NCEP', 'JMA', 'ECCC']

### get today's date 

In [8]:
date = datetime.utcnow()

### apply lag 

In [9]:
date = date - relativedelta(months=lag)

In [10]:
print(f"will process forecasts issued in {date:%B %Y}")

will process forecasts issued in September 2021


In [11]:
year = date.year 

In [12]:
year 

2021

In [13]:
month = date.month 

In [14]:
month

9

### list files for the individual GCMs

In [15]:
opath = CWD.parents[1].joinpath("outputs/C3S")

In [16]:
opath = opath.joinpath(f'admin/{quantiles}')

In [17]:
lfiles = list(opath.glob(f"{quantiles}_probs_forecasts_from_{year}_{str(month).zfill(2)}_{utils.sanitize_name(country)}_{utils.sanitize_name(admin)}_*.csv"))

In [18]:
len(lfiles)

7

In [19]:
df_all = []
for fname in lfiles: 
    df = pd.read_csv(fname, header=[0,1], index_col=[0,1,2,3])
    df_all.append(df)

In [20]:
df = pd.concat(df_all, axis=0)

In [21]:
step = 3

In [22]:
df = df.xs(str(step), axis=1, level=0)

In [23]:
df

tercile                                                  1          2  \
2021-09-01 Kiribati Gilberts-North DWD           64.000000  36.000000   
                                   NCEP          94.354839   4.838710   
                                   CMCC          56.000000  44.000000   
                                   ECMWF         84.313725  15.686275   
                                   UKMO          93.548387   4.838710   
                                   JMA           66.666667  33.333333   
                                   METEO_FRANCE  84.313725  15.686275   

tercile                                                 3  
2021-09-01 Kiribati Gilberts-North DWD           0.000000  
                                   NCEP          0.806452  
                                   CMCC          0.000000  
                                   ECMWF         0.000000  
                                   UKMO          1.612903  
                                   JMA           0.000000  
                                   METEO_FRANCE  0.000000

In [24]:
df.index[0]

('2021-09-01', 'Kiribati', 'Gilberts-North', 'DWD')

In [25]:
tuple(list(df.index[-1][:-1]) + ['MME'])

('2021-09-01', 'Kiribati', 'Gilberts-North', 'MME')

In [26]:
df.loc[tuple(list(df.index[-1][:-1]) + ['MME']),:] = df.mean(0)

In [27]:
df

tercile                                                  1          2  \
2021-09-01 Kiribati Gilberts-North DWD           64.000000  36.000000   
                                   NCEP          94.354839   4.838710   
                                   CMCC          56.000000  44.000000   
                                   ECMWF         84.313725  15.686275   
                                   UKMO          93.548387   4.838710   
                                   JMA           66.666667  33.333333   
                                   METEO_FRANCE  84.313725  15.686275   
                                   MME           77.599620  22.054757   

tercile                                                 3  
2021-09-01 Kiribati Gilberts-North DWD           0.000000  
                                   NCEP          0.806452  
                                   CMCC          0.000000  
                                   ECMWF         0.000000  
                                   UKMO          1.612903  
                                   JMA           0.000000  
                                   METEO_FRANCE  0.000000  
                                   MME           0.345622

In [28]:
df = df.T.apply(utils.round_to_100_percent, **{'digit_after_decimal':0}).T.astype(int)